In [1]:
import squidpy as sq
import scanpy as sc

from numpy.random import default_rng
from anndata import AnnData
import scipy
from sklearn.metrics.cluster import adjusted_rand_score
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn import metrics
import multiprocessing as mp
from sklearn.metrics.cluster import adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score
import pickle
from sklearn.neighbors import kneighbors_graph
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture

In [2]:
from numpy import genfromtxt
import time

In [3]:
from SpaceFlow import SpaceFlow

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /data/hoan/mybin/miniconda3/envs/spaceflow/lib/python3.7/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /data/hoan/mybin/miniconda3/envs/spaceflow/lib/python3.7/site-packages/torch_sparse/_version_cpu.so:

In [4]:
# adata = sq.datasets.seqfish()
# sc.pp.filter_genes(adata, min_cells=3)
# sf = SpaceFlow.SpaceFlow(adata=adata)
# sf.preprocessing_data(n_top_genes=3000)
# sf.train(spatial_regularization_strength=0.1, z_dim=50, lr=1e-3, epochs=30, max_patience=50, min_stop=100, random_seed=42, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)
# sf.segmentation(domain_label_save_filepath="output/domains.tsv", n_neighbors=50, resolution=1.0)

## MsMERFISH

In [18]:
data_dir = '/data/hoan/spatial_transcriptomics/data/SDMBench/Data/378k/BrainAgingSpatialAtlas_MERFISH.h5ad'
adata_big=sc.read_h5ad(data_dir)
donor_id = np.unique(adata_big.obs['donor_id'])

In [121]:
sf_embedding_all = np.array([], dtype=np.int64).reshape(0,50)
batch_labels = []
batch_index = 0
sf_meta = []
for sliceID in ['0', '1', '2']:
    for donorID in donor_id:
# sliceID = '1'; donorID = donor_id[2]; #1, 2
# for sliceID in ['0', '1', '2']:
        print(sliceID, donorID)
        adata = adata_big[adata_big.obs['slice']==sliceID]#[:5000]
        adata = adata[adata.obs['donor_id']==donorID]
        adata.obsm['spatial'] = adata.obsm['spatial_coords']
        true_labels = pd.factorize(adata.obs['tissue'])[0]
        if adata.shape[0] > 10:
            time_st = time.time()
            data_name = donorID + 'Slice' + sliceID
            adataNew = AnnData(1.0 + adata.obsm['X_pca'] - adata.obsm['X_pca'].min())
            adataNew.obsm['spatial'] = adata.obsm['spatial']
            sf = SpaceFlow.SpaceFlow(adata=adataNew)
            sf.preprocessing_data(n_top_genes=40)
            embed_filepath = 'output/'+data_name+'SpaceFlowembedding.tsv'
            domain_label = 'output/'+data_name+'SpaceFlowdomains.tsv'
            sf.train(embedding_save_filepath=embed_filepath, spatial_regularization_strength=0.1, z_dim=50, lr=1e-3, epochs=2, max_patience=50, min_stop=100, random_seed=42, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)
            
            # sf.segmentation(domain_label_save_filepath=domain_label, n_neighbors=50, resolution=1.0)
            sf_embedding = genfromtxt(embed_filepath, delimiter='\t')
            embedding_adata = AnnData(sf_embedding)
            sc.pp.neighbors(embedding_adata, n_neighbors=50, use_rep='X')
            sc.tl.leiden(embedding_adata, resolution=1.0)
            domains = embedding_adata.obs["leiden"].cat.codes
            time_ed = time.time()
            time_spaceflow = time_ed-time_st

            # cluster_label = pd.read_csv(domain_label, header = None)
            # cluster_SpaceFlow = cluster_label.values.flatten()
            cluster_SpaceFlow = np.array(domains)
            df_out = pd.DataFrame()
            df_out['SpaceFLow'] = np.array(cluster_SpaceFlow[true_labels>=0]).astype(int)
            df_out['ground_truth'] = np.array(true_labels[true_labels>=0]).astype(int)
            out_data = '/data/hoan/spatial_transcriptomics/GraphST/output/'+data_name+'SpaceFlow.csv'
            df_out.to_csv(out_data, index=False)
            time_st = time.time()
            sf_embedding = genfromtxt(embed_filepath, delimiter='\t')
            sf_embedding_all = np.vstack([sf_embedding_all, sf_embedding])
            batch_labels += [batch_index]*len(cluster_SpaceFlow)
            batch_index += 1
            time_ed = time.time()
            time_combine_preprocess = time_ed-time_st
            sf_info = [data_name, len(cluster_SpaceFlow), normalized_mutual_info_score(true_labels, cluster_SpaceFlow), time_spaceflow, time_combine_preprocess]
            sf_meta.append(sf_info)
            break;

0 MsBrainAgingSpatialDonor_1
Epoch 2/2, Loss: 1.8738986253738403
Training complete!
Embedding is saved at output/MsBrainAgingSpatialDonor_1Slice0SpaceFlowembedding.tsv


/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


1 MsBrainAgingSpatialDonor_1
1 MsBrainAgingSpatialDonor_10
Epoch 2/2, Loss: 4.892630577087402
Training complete!
Embedding is saved at output/MsBrainAgingSpatialDonor_10Slice1SpaceFlowembedding.tsv


/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


2 MsBrainAgingSpatialDonor_1
2 MsBrainAgingSpatialDonor_10
Epoch 2/2, Loss: 2.943315029144287
Training complete!
Embedding is saved at output/MsBrainAgingSpatialDonor_10Slice2SpaceFlowembedding.tsv


/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


In [123]:
sf_meta_df = pd.DataFrame(sf_meta, columns = ['metaSlice', 'n', 'NMI', 'SpaceFlowRuntime', 'Hamony_prepro_runtime'])
sf_meta_df.to_csv('sf_meta_df_runtime_round1.csv')

In [124]:
sf_meta_df

,metaSlice,n,NMI,SpaceFlowRuntime,Hamony_prepro_runtime
0,MsBrainAgingSpatialDonor_1Slice0,21002,0.168404,47.137085,1.041497
1,MsBrainAgingSpatialDonor_10Slice1,9617,0.264481,14.025429,0.420151
2,MsBrainAgingSpatialDonor_10Slice2,8823,0.257776,16.631531,0.372299


In [111]:
import scanpy.external as sce
time_st = time.time()
adata_combine = sc.AnnData(sf_embedding_all)
adata_combine.obsm['X_pca'] = sf_embedding_all
adata_combine.obs["batch"] = np.array(batch_labels)
adata_combine.obs["batch"] = adata_combine.obs["batch"].astype("category")
# sc.pp.combat(adata_combine, key="batch", inplace=True)
# sce.pp.scanorama_integrate(adata, key="batch")
# sc.pp.pca(adata_combine, n_comps=30)
sce.pp.harmony_integrate(adata_combine, key="batch")
time_ed = time.time()
time_harmony = time_ed-time_st

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  This is separate from the ipykernel package so we can avoid doing imports until
2025-02-09 15:33:21,332 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2025-02-09 15:34:05,193 - harmonypy - INFO - sklearn.KMeans initialization complete.
2025-02-09 15:34:09,880 - harmonypy - INFO - Iteration 1 of 10
2025-02-09 15:36:05,014 - harmonypy - INFO - Iteration 2 of 10
2025-02-09 15:39:47,923 - harmonypy - INFO - Iteration 3 of 10
2025-02-09 15:42:06,164 - harmonypy - INFO - Iteration 4 of 10
2025-02-09 15:43:59,291 - harmonypy - INFO - Iteration 5 of 10
2025-02-09 15:45:52,706 - harmonypy - INFO - Iteration 6 of 10
2025-02-09 15:

In [112]:
time_st = time.time()
sc.pp.neighbors(adata_combine,use_rep='X_pca_harmony')
sc.tl.leiden(adata_combine,resolution=0.5)
time_ed = time.time()
time_leiden = time_ed-time_st

In [116]:
sf_meta_df['Harmony_combine_Time'] = time_harmony
sf_meta_df['time_leiden'] =time_leiden
sf_meta_df.to_csv('sf_meta_df_runtime.csv')

# Small MERFISH

In [4]:
# data_dir = '/data/hoan/spatial_transcriptomics/data/SDMBench/Data/378k/BrainAgingSpatialAtlas_MERFISH.h5ad'
# adata_big=sc.read_h5ad(data_dir)
# donor_id = np.unique(adata_big.obs['donor_id'])

In [5]:
import glob
data_dir_full = []
for data_dir in glob.glob("/data/hoan/spatial_transcriptomics/data/SDMBench/Data/MERFISH/*.h5ad", recursive=True):
    # print(data_dir)
    data_dir_full.append(data_dir)
# data_dir_full = data_dir_full[-4:]

In [27]:
sf_embedding_all = np.array([], dtype=np.int64).reshape(0,50)
batch_labels = []
batch_index = 0
sf_meta = []
adata_list = []
# data_dir_full = data_dir_full[1:3]
combined_labels = []
for data_dir in data_dir_full:
    data_name = data_dir.split('/')[-2]+data_dir.split('/')[-1][:-5]
    # file_pickle = '/data/hoan/spatial_transcriptomics/GraphBGM/Data/Pickle/'+data_name+'.pickle'
    # if os.path.exists(file_pickle):
    # for data_dir in data_dir_full:
    adata = sc.read_h5ad(data_dir)
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
    # sc.pp.highly_variable_genes(adata,  n_top_genes=3000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.scale(adata, zero_center=False, max_value=10)
    # Reload the file
    # adata = pickle.load(open(file_pickle, "rb"))
    true_labels = pd.factorize(adata.obs['Region'])[0]
    combined_labels += list(true_labels)
    if adata.shape[0] > 10:
        time_st = time.time()
        # data_name = donorID + 'Slice' + sliceID
        # adataNew = AnnData(1.0 + adata.obsm['X_pca'] - adata.obsm['X_pca'].min())
        # adataNew.obsm['spatial'] = adata.obsm['spatial']
        #sf = SpaceFlow.SpaceFlow(adata=adataNew)
        sf = SpaceFlow.SpaceFlow(adata=adata)
        # sf.preprocessing_data(n_top_genes=40) # For big MERFISH dataset
        sf.preprocessing_data(n_top_genes=3000) 
        embed_filepath = 'output/'+data_name+'SpaceFlowembedding.tsv'
        domain_label = 'output/'+data_name+'SpaceFlowdomains.tsv'
        sf.train(embedding_save_filepath=embed_filepath, spatial_regularization_strength=0.1, z_dim=50, lr=1e-3, epochs=300, max_patience=50, min_stop=100, random_seed=42, gpu=0, regularization_acceleration=True, edge_subset_sz=1000000)

        # sf.segmentation(domain_label_save_filepath=domain_label, n_neighbors=50, resolution=1.0)
        # sf_embedding = genfromtxt(embed_filepath, delimiter='\t')
        # embedding_adata = AnnData(sf_embedding)
#         sc.pp.neighbors(embedding_adata, n_neighbors=50, use_rep='X')
#         sc.tl.leiden(embedding_adata, resolution=1.0)
#         domains = embedding_adata.obs["leiden"].cat.codes
#         time_ed = time.time()
#         time_spaceflow = time_ed-time_st

#         # cluster_label = pd.read_csv(domain_label, header = None)
#         # cluster_SpaceFlow = cluster_label.values.flatten()
#         cluster_SpaceFlow = np.array(domains)
#         df_out = pd.DataFrame()
#         df_out['SpaceFLow'] = np.array(cluster_SpaceFlow[true_labels>=0]).astype(int)
#         df_out['ground_truth'] = np.array(true_labels[true_labels>=0]).astype(int)
#         out_data = '/data/hoan/spatial_transcriptomics/GraphST/output/'+data_name+'SpaceFlow.csv'
#         df_out.to_csv(out_data, index=False)
        time_st = time.time()
        sf_embedding = genfromtxt(embed_filepath, delimiter='\t')
        sf_embedding_all = np.vstack([sf_embedding_all, sf_embedding])
        batch_labels += [batch_index]*int(adata.shape[0])
        batch_index += 1
        time_ed = time.time()
        time_combine_preprocess = time_ed-time_st
        # sf_info = [data_name, len(cluster_SpaceFlow), normalized_mutual_info_score(true_labels, cluster_SpaceFlow), time_spaceflow, time_combine_preprocess]
        # sf_meta.append(sf_info)
combined_labels = np.array(combined_labels)

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


Epoch 2/300, Loss: 3.014679431915283
Epoch 12/300, Loss: 1.627250075340271
Epoch 22/300, Loss: 1.4462496042251587
Epoch 32/300, Loss: 1.3566622734069824
Epoch 42/300, Loss: 1.290056586265564
Epoch 52/300, Loss: 1.2278144359588623
Epoch 62/300, Loss: 1.1679853200912476
Epoch 72/300, Loss: 1.1063367128372192
Epoch 82/300, Loss: 1.0538140535354614
Epoch 92/300, Loss: 0.9905679821968079
Epoch 102/300, Loss: 0.9558861255645752
Epoch 112/300, Loss: 0.8885167837142944
Epoch 122/300, Loss: 0.849240243434906
Epoch 132/300, Loss: 0.8113704919815063
Epoch 142/300, Loss: 0.7240058183670044
Epoch 152/300, Loss: 0.7023977637290955
Epoch 162/300, Loss: 0.6468055844306946
Epoch 172/300, Loss: 0.5882895588874817
Epoch 182/300, Loss: 0.5435284376144409
Epoch 192/300, Loss: 0.5278501510620117
Epoch 202/300, Loss: 0.514238715171814
Epoch 212/300, Loss: 0.4841987192630768
Epoch 222/300, Loss: 0.43486547470092773
Epoch 232/300, Loss: 0.42349690198898315
Epoch 242/300, Loss: 0.4104222357273102
Epoch 252/300,

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


Epoch 2/300, Loss: 2.462718963623047
Epoch 12/300, Loss: 1.6010754108428955
Epoch 22/300, Loss: 1.4592583179473877
Epoch 32/300, Loss: 1.3873071670532227
Epoch 42/300, Loss: 1.3325777053833008
Epoch 52/300, Loss: 1.2597651481628418
Epoch 62/300, Loss: 1.1943200826644897
Epoch 72/300, Loss: 1.123783826828003
Epoch 82/300, Loss: 1.0655163526535034
Epoch 92/300, Loss: 0.9882709980010986
Epoch 102/300, Loss: 0.9499662518501282
Epoch 112/300, Loss: 0.8969447612762451
Epoch 122/300, Loss: 0.8437889814376831
Epoch 132/300, Loss: 0.7921134233474731
Epoch 142/300, Loss: 0.7482636570930481
Epoch 152/300, Loss: 0.6999286413192749
Epoch 162/300, Loss: 0.6419752240180969
Epoch 172/300, Loss: 0.6081466674804688
Epoch 182/300, Loss: 0.5803207159042358
Epoch 192/300, Loss: 0.5243596434593201
Epoch 202/300, Loss: 0.5146520137786865
Epoch 212/300, Loss: 0.4696471691131592
Epoch 222/300, Loss: 0.43562400341033936
Epoch 232/300, Loss: 0.43749672174453735
Epoch 242/300, Loss: 0.37804239988327026
Epoch 252/

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


Epoch 2/300, Loss: 2.6842925548553467
Epoch 12/300, Loss: 1.6088272333145142
Epoch 22/300, Loss: 1.447527289390564
Epoch 32/300, Loss: 1.345799207687378
Epoch 42/300, Loss: 1.2836519479751587
Epoch 52/300, Loss: 1.2209776639938354
Epoch 62/300, Loss: 1.1573399305343628
Epoch 72/300, Loss: 1.096838116645813
Epoch 82/300, Loss: 1.0332142114639282
Epoch 92/300, Loss: 0.974266767501831
Epoch 102/300, Loss: 0.931865930557251
Epoch 112/300, Loss: 0.863125205039978
Epoch 122/300, Loss: 0.8068112730979919
Epoch 132/300, Loss: 0.7512430548667908
Epoch 142/300, Loss: 0.6971694231033325
Epoch 152/300, Loss: 0.6239796876907349
Epoch 162/300, Loss: 0.5911360383033752
Epoch 172/300, Loss: 0.5475049018859863
Epoch 182/300, Loss: 0.5194346308708191
Epoch 192/300, Loss: 0.46391725540161133
Epoch 202/300, Loss: 0.408311665058136
Epoch 212/300, Loss: 0.38679590821266174
Epoch 222/300, Loss: 0.35682088136672974
Epoch 232/300, Loss: 0.3489559590816498
Epoch 242/300, Loss: 0.3209990859031677
Epoch 252/300, 

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


Epoch 2/300, Loss: 2.3171207904815674
Epoch 12/300, Loss: 1.6099742650985718
Epoch 22/300, Loss: 1.4566670656204224
Epoch 32/300, Loss: 1.3760632276535034
Epoch 42/300, Loss: 1.3321751356124878
Epoch 52/300, Loss: 1.2737704515457153
Epoch 62/300, Loss: 1.2091922760009766
Epoch 72/300, Loss: 1.150964379310608
Epoch 82/300, Loss: 1.0932539701461792
Epoch 92/300, Loss: 1.034947156906128
Epoch 102/300, Loss: 0.9509592056274414
Epoch 112/300, Loss: 0.8865514397621155
Epoch 122/300, Loss: 0.8284448981285095
Epoch 132/300, Loss: 0.7662526369094849
Epoch 142/300, Loss: 0.7130587100982666
Epoch 152/300, Loss: 0.6773752570152283
Epoch 162/300, Loss: 0.6127500534057617
Epoch 172/300, Loss: 0.5671331882476807
Epoch 182/300, Loss: 0.5381788015365601
Epoch 192/300, Loss: 0.5186752080917358
Epoch 202/300, Loss: 0.4928723871707916
Epoch 212/300, Loss: 0.4584631621837616
Epoch 222/300, Loss: 0.42092469334602356
Epoch 232/300, Loss: 0.4223036766052246
Epoch 242/300, Loss: 0.3840133249759674
Epoch 252/30

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  UserWarning,


Epoch 2/300, Loss: 2.633160352706909
Epoch 12/300, Loss: 1.5779093503952026
Epoch 22/300, Loss: 1.429612159729004
Epoch 32/300, Loss: 1.3644073009490967
Epoch 42/300, Loss: 1.3060466051101685
Epoch 52/300, Loss: 1.236332654953003
Epoch 62/300, Loss: 1.175241470336914
Epoch 72/300, Loss: 1.1133692264556885
Epoch 82/300, Loss: 1.0353931188583374
Epoch 92/300, Loss: 0.9569563865661621
Epoch 102/300, Loss: 0.9092714190483093
Epoch 112/300, Loss: 0.8352928161621094
Epoch 122/300, Loss: 0.7739911675453186
Epoch 132/300, Loss: 0.6943329572677612
Epoch 142/300, Loss: 0.642557680606842
Epoch 152/300, Loss: 0.6370167136192322
Epoch 162/300, Loss: 0.5712909698486328
Epoch 172/300, Loss: 0.5096271634101868
Epoch 182/300, Loss: 0.47604379057884216
Epoch 192/300, Loss: 0.46454480290412903
Epoch 202/300, Loss: 0.40800604224205017
Epoch 212/300, Loss: 0.3978159725666046
Epoch 222/300, Loss: 0.3980894684791565
Epoch 232/300, Loss: 0.3602585196495056
Epoch 242/300, Loss: 0.33001941442489624
Epoch 252/30

In [28]:
adata

AnnData object with n_obs × n_vars = 5543 × 155
    obs: 'cell_class', 'neuron_class', 'domain', 'Region'
    var: 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'mean', 'std', 'dispersions', 'dispersions_norm'
    uns: 'domain_colors', 'hvg', 'log1p', 'pca'
    obsm: 'spatial', 'X_pca'
    varm: 'PCs'

In [29]:
# sf_meta_df = pd.DataFrame(sf_meta, columns = ['metaSlice', 'n', 'NMI', 'SpaceFlowRuntime', 'Hamony_prepro_runtime'])
# sf_meta_df.to_csv('sf_meta_df_runtime_round1.csv')
# sf_meta_df

In [30]:
import scanpy.external as sce
time_st = time.time()
adata_combine = sc.AnnData(sf_embedding_all)
adata_combine.obsm['X_pca'] = sf_embedding_all
adata_combine.obs["batch"] = np.array(batch_labels)
adata_combine.obs["batch"] = adata_combine.obs["batch"].astype("category")
# sc.pp.combat(adata_combine, key="batch", inplace=True)
# sce.pp.scanorama_integrate(adata, key="batch")
# sc.pp.pca(adata_combine, n_comps=30)
sce.pp.harmony_integrate(adata_combine, key="batch")
time_ed = time.time()
time_harmony = time_ed-time_st

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  This is separate from the ipykernel package so we can avoid doing imports until
2025-03-28 03:26:54,744 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...
2025-03-28 03:27:06,260 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2025-03-28 03:27:06,665 - harmonypy - INFO - Iteration 1 of 10
INFO:harmonypy:Iteration 1 of 10
2025-03-28 03:27:26,434 - harmonypy - INFO - Iteration 2 of 10
INFO:harmonypy:Iteration 2 of 10
2025-03-28 03:27:34,877 - harmonypy - INFO - Iteration 3 of 10
INFO:harmonypy:I

In [31]:
time_st = time.time()
sc.pp.neighbors(adata_combine,use_rep='X_pca_harmony')
sc.tl.leiden(adata_combine,resolution=0.5)
time_ed = time.time()
time_leiden = time_ed-time_st

In [32]:
# sf_meta_df['Harmony_combine_Time'] = time_harmony
# sf_meta_df['time_leiden'] =time_leiden
# sf_meta_df.to_csv('sf_meta_df_runtime.csv')

In [33]:
adata_combine
pred_labels = adata_combine.obs['leiden']
pred_labels = np.array(pred_labels) #v1
print(round(normalized_mutual_info_score(combined_labels[combined_labels>0], pred_labels[combined_labels>0]), 6),
     round(homogeneity_score(combined_labels[combined_labels>0], pred_labels[combined_labels>0]), 6),
     round(completeness_score(combined_labels[combined_labels>0], pred_labels[combined_labels>0]), 6))
# 0.231346 0.262239 0.206965 epoch = 300
# 0.248798 0.28346 0.221689 #epoch = 1000
# 0.316288 0.381606 0.270063
# 0.295028 0.356793 0.251491 # genes = 3000, epoch = 300
# 0.332379 0.418154 0.275804 # g = 3000, epoch = 300, preprocessing


0.332379 0.418154 0.275804


In [34]:
from sklearn import preprocessing
def eval_leiden(adata, resol):
    sc.tl.leiden(adata, key_added='louvain', resolution = resol)
    louv_labels = np.array(adata.obs['louvain'].tolist())

    le = preprocessing.LabelEncoder().fit(louv_labels)
    cell_labels_pred  = le.transform(louv_labels)
    return int(np.unique(cell_labels_pred).shape[0])

def eval_bisect_leiden(adata, minresolution, maxresolution, n_clusters):
    M = -1
    k1 = n_clusters + 1
    k2 = 0
    iterations = 0 
    #adata = AnnData(X = data)
    #sc.pp.neighbors(adata, n_neighbors=20, use_rep='X')

    # find minresolution and maxresolution s.t k1 < n_clusters < k2 
    while k1 > n_clusters and iterations < 8:
        minresolution = minresolution/2 
        k1 = eval_leiden(adata, minresolution)
        if k1 == n_clusters:
            return minresolution
        iterations = iterations + 1
    while k2 < n_clusters and iterations < 8:
        maxresolution = 2*maxresolution
        k2 = eval_leiden(adata, maxresolution)
        if k2 == n_clusters:
            return maxresolution
        iterations = iterations + 1

    # bisection main
    while iterations < 40 and abs(maxresolution - minresolution) > 1e-5:
        M = (minresolution + maxresolution)/2 
        iterations = iterations + 1
        k3 = eval_leiden(adata, M)
        if k3 == n_clusters:
            return M 
        elif k3 < n_clusters:
            minresolution = M 
        else:
            maxresolution = M

    if iterations >= 40:
        print("bisection algorithm could not find the right resolution")

def leiden_exact_K(X_dimred, n_clusters):
    adata = AnnData(X=X_dimred)
    sc.pp.neighbors(adata,n_neighbors=10, use_rep='X')

    resol = eval_bisect_leiden(adata, 0.5, 1.7, n_clusters)

    sc.tl.leiden(adata, key_added='louvain', resolution = resol)
    louv_labels = np.array(adata.obs['louvain'].tolist())
    le = preprocessing.LabelEncoder().fit(louv_labels)
    cell_labels_pred  = le.transform(louv_labels)
    return cell_labels_pred

In [ ]:
n_clusters = len(set(combined_labels))
pred_exact_labels = leiden_exact_K(adata_combine.obsm['X_pca_harmony'], n_clusters)

/home/vanhoan310/miniconda3/envs/spaceflow/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


In [ ]:
print(round(normalized_mutual_info_score(combined_labels[combined_labels>0], pred_exact_labels[combined_labels>0]), 6),
     round(homogeneity_score(combined_labels[combined_labels>0], pred_exact_labels[combined_labels>0]), 6),
     round(completeness_score(combined_labels[combined_labels>0], pred_exact_labels[combined_labels>0]), 6))

In [ ]:
np.unique(pred_labels), n_clusters